# 📊 GetAround - Analyse des Retards

## Objectif
Analyser les retards au checkout et déterminer le seuil optimal de délai minimum entre deux locations.

## Questions clés à répondre
1. Quelle part du revenu des propriétaires serait affectée par le délai minimum ?
2. Combien de locations seraient impactées selon le seuil choisi ?
3. À quelle fréquence les conducteurs sont-ils en retard ?
4. Combien de cas problématiques seraient résolus selon le seuil ?

## 1. Chargement des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Taille des figures
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Bibliothèques chargées avec succès")

✅ Bibliothèques chargées avec succès


## 2. Chargement des données

In [2]:
# Charger le fichier Excel
df_delays = pd.read_excel('../data/get_around_delay_analysis.xlsx')

print(f"📊 Données chargées : {len(df_delays):,} lignes et {len(df_delays.columns)} colonnes")
print(f"\n🗓️ Période des données : {df_delays.index.min()} à {df_delays.index.max()}" if df_delays.index.name else "")

📊 Données chargées : 21,310 lignes et 7 colonnes



## 3. Exploration initiale

In [3]:
# Aperçu des données
print("="*80)
print("APERÇU DES DONNÉES")
print("="*80)
df_delays.head(10)

APERÇU DES DONNÉES


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0
7,512303,371242,mobile,ended,-44.0,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN


In [4]:
# Informations sur les colonnes
print("="*80)
print("INFORMATIONS SUR LES COLONNES")
print("="*80)
df_delays.info()

INFORMATIONS SUR LES COLONNES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB


In [5]:
# Statistiques descriptives
print("="*80)
print("STATISTIQUES DESCRIPTIVES")
print("="*80)
df_delays.describe()

STATISTIQUES DESCRIPTIVES


,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000


In [6]:
# Valeurs manquantes
print("="*80)
print("VALEURS MANQUANTES")
print("="*80)
missing = df_delays.isnull().sum()
missing_pct = (missing / len(df_delays) * 100).round(2)
missing_df = pd.DataFrame({
    'Nombre': missing,
    'Pourcentage': missing_pct
})
print(missing_df[missing_df['Nombre'] > 0].sort_values('Nombre', ascending=False))

VALEURS MANQUANTES
                                            Nombre  Pourcentage
previous_ended_rental_id                     19469        91.36
time_delta_with_previous_rental_in_minutes   19469        91.36
delay_at_checkout_in_minutes                  4964        23.29


## 4. Analyse des retards

### 4.1 Distribution générale des retards

In [7]:
# Métriques globales sur les retards
print("="*80)
print("📊 MÉTRIQUES GLOBALES - RETARDS")
print("="*80)

total_rentals = len(df_delays)
late_rentals = (df_delays['delay_at_checkout_in_minutes'] > 0).sum()
on_time_rentals = total_rentals - late_rentals
late_pct = (late_rentals / total_rentals * 100)

print(f"\n📍 Total de locations : {total_rentals:,}")
print(f"✅ Locations à l'heure : {on_time_rentals:,} ({100-late_pct:.1f}%)")
print(f"⏰ Locations en retard : {late_rentals:,} ({late_pct:.1f}%)")

if late_rentals > 0:
    avg_delay = df_delays[df_delays['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean()
    median_delay = df_delays[df_delays['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].median()
    max_delay = df_delays['delay_at_checkout_in_minutes'].max()
    
    print(f"\n📈 Statistiques des retards :")
    print(f"   - Retard moyen : {avg_delay:.1f} minutes ({avg_delay/60:.1f}h)")
    print(f"   - Retard médian : {median_delay:.1f} minutes ({median_delay/60:.1f}h)")
    print(f"   - Retard maximum : {max_delay:.1f} minutes ({max_delay/60:.1f}h)")

📊 MÉTRIQUES GLOBALES - RETARDS

📍 Total de locations : 21,310
✅ Locations à l'heure : 11,906 (55.9%)
⏰ Locations en retard : 9,404 (44.1%)

📈 Statistiques des retards :
   - Retard moyen : 201.8 minutes (3.4h)
   - Retard médian : 53.0 minutes (0.9h)
   - Retard maximum : 71084.0 minutes (1184.7h)


In [8]:
# Graphique : Distribution des retards
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribution des retards', 'Proportion retard vs à l\'heure'),
    specs=[[{'type': 'histogram'}, {'type': 'pie'}]]
)

# Histogramme
fig.add_trace(
    go.Histogram(
        x=df_delays['delay_at_checkout_in_minutes'],
        nbinsx=50,
        name='Retards',
        marker_color='indianred'
    ),
    row=1, col=1
)

# Pie chart
fig.add_trace(
    go.Pie(
        labels=['À l\'heure', 'En retard'],
        values=[on_time_rentals, late_rentals],
        marker_colors=['lightgreen', 'indianred']
    ),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, title_text="Vue d'ensemble des retards")
fig.show()

### 4.2 Retards par type de checkin

In [9]:
# Analyse par type de checkin
print("="*80)
print("📱 RETARDS PAR TYPE DE CHECKIN")
print("="*80)

checkin_analysis = df_delays.groupby('checkin_type').agg({
    'rental_id': 'count',
    'delay_at_checkout_in_minutes': ['mean', 'median']
})

# Calculer le pourcentage de retards par type
for checkin_type in df_delays['checkin_type'].unique():
    if pd.notna(checkin_type):
        subset = df_delays[df_delays['checkin_type'] == checkin_type]
        late_count = (subset['delay_at_checkout_in_minutes'] > 0).sum()
        late_pct = (late_count / len(subset) * 100)
        print(f"\n{checkin_type}:")
        print(f"  Total locations : {len(subset):,}")
        print(f"  En retard : {late_count:,} ({late_pct:.1f}%)")
        if late_count > 0:
            avg = subset[subset['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean()
            print(f"  Retard moyen : {avg:.1f} min")

📱 RETARDS PAR TYPE DE CHECKIN

mobile:
  Total locations : 17,003
  En retard : 7,945 (46.7%)
  Retard moyen : 224.1 min

connect:
  Total locations : 4,307
  En retard : 1,459 (33.9%)
  Retard moyen : 80.1 min


In [10]:
# Graphiques comparatifs par type
checkin_stats = []
for checkin_type in df_delays['checkin_type'].dropna().unique():
    subset = df_delays[df_delays['checkin_type'] == checkin_type]
    late_count = (subset['delay_at_checkout_in_minutes'] > 0).sum()
    late_pct = (late_count / len(subset) * 100)
    avg_delay = subset[subset['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean() if late_count > 0 else 0
    
    checkin_stats.append({
        'Type': checkin_type,
        'Total': len(subset),
        'Retards': late_count,
        'Pct_retards': late_pct,
        'Retard_moyen': avg_delay
    })

df_checkin = pd.DataFrame(checkin_stats)

# Graphiques
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('% de retards par type', 'Retard moyen par type (minutes)')
)

fig.add_trace(
    go.Bar(x=df_checkin['Type'], y=df_checkin['Pct_retards'], name='% retards', marker_color='coral'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=df_checkin['Type'], y=df_checkin['Retard_moyen'], name='Retard moyen', marker_color='skyblue'),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False)
fig.show()

### 4.3 Impact sur les locations suivantes

In [11]:
# Filtrer les locations avec une location suivante
df_with_next = df_delays[df_delays['time_delta_with_previous_rental_in_minutes'].notna()].copy()

print("="*80)
print("🔗 IMPACT SUR LES LOCATIONS SUIVANTES")
print("="*80)
print(f"\nLocations avec location suivante : {len(df_with_next):,}")
print(f"Pourcentage du total : {len(df_with_next)/len(df_delays)*100:.1f}%")

# Identifier les cas problématiques
df_with_next['is_problematic'] = (
    (df_with_next['delay_at_checkout_in_minutes'] > 0) & 
    (df_with_next['delay_at_checkout_in_minutes'] > df_with_next['time_delta_with_previous_rental_in_minutes'])
)

total_problems = df_with_next['is_problematic'].sum()
problem_pct = (total_problems / len(df_with_next) * 100)

print(f"\n⚠️ Cas problématiques (retard > délai entre locations) :")
print(f"   Nombre : {total_problems:,}")
print(f"   Pourcentage : {problem_pct:.2f}%")
print(f"\n💡 Ces cas représentent des situations où le client suivant a été impacté")

🔗 IMPACT SUR LES LOCATIONS SUIVANTES

Locations avec location suivante : 1,841
Pourcentage du total : 8.6%

⚠️ Cas problématiques (retard > délai entre locations) :
   Nombre : 270
   Pourcentage : 14.67%

💡 Ces cas représentent des situations où le client suivant a été impacté


## 5. Simulation de seuils

### 5.1 Impact de différents seuils

In [12]:
# Définir les seuils à tester
thresholds = [0, 30, 60, 120, 180, 240, 360, 480, 720]  # en minutes

print("="*80)
print("🎯 SIMULATION DE DIFFÉRENTS SEUILS")
print("="*80)

results = []

for threshold in thresholds:
    # Locations qui seraient bloquées
    blocked = df_with_next[df_with_next['time_delta_with_previous_rental_in_minutes'] < threshold]
    blocked_count = len(blocked)
    blocked_pct = (blocked_count / len(df_with_next) * 100)
    
    # Problèmes résolus
    problems_solved = df_with_next[
        (df_with_next['is_problematic']) & 
        (df_with_next['time_delta_with_previous_rental_in_minutes'] < threshold)
    ]
    solved_count = len(problems_solved)
    solved_pct = (solved_count / total_problems * 100) if total_problems > 0 else 0
    
    results.append({
        'Seuil_min': threshold,
        'Seuil_h': threshold / 60,
        'Locations_bloquees': blocked_count,
        'Pct_bloquees': blocked_pct,
        'Problemes_resolus': solved_count,
        'Pct_resolus': solved_pct
    })
    
    print(f"\nSeuil : {threshold:3d} min ({threshold/60:5.1f}h)")
    print(f"  Locations bloquées : {blocked_count:5,} ({blocked_pct:5.1f}%)")
    print(f"  Problèmes résolus  : {solved_count:5,} ({solved_pct:5.1f}%)")

df_results = pd.DataFrame(results)

🎯 SIMULATION DE DIFFÉRENTS SEUILS

Seuil :   0 min (  0.0h)
  Locations bloquées :     0 (  0.0%)
  Problèmes résolus  :     0 (  0.0%)

Seuil :  30 min (  0.5h)
  Locations bloquées :   279 ( 15.2%)
  Problèmes résolus  :   136 ( 50.4%)

Seuil :  60 min (  1.0h)
  Locations bloquées :   401 ( 21.8%)
  Problèmes résolus  :   176 ( 65.2%)

Seuil : 120 min (  2.0h)
  Locations bloquées :   666 ( 36.2%)
  Problèmes résolus  :   224 ( 83.0%)

Seuil : 180 min (  3.0h)
  Locations bloquées :   870 ( 47.3%)
  Problèmes résolus  :   245 ( 90.7%)

Seuil : 240 min (  4.0h)
  Locations bloquées : 1,001 ( 54.4%)
  Problèmes résolus  :   251 ( 93.0%)

Seuil : 360 min (  6.0h)
  Locations bloquées : 1,170 ( 63.6%)
  Problèmes résolus  :   255 ( 94.4%)

Seuil : 480 min (  8.0h)
  Locations bloquées : 1,290 ( 70.1%)
  Problèmes résolus  :   259 ( 95.9%)

Seuil : 720 min ( 12.0h)
  Locations bloquées : 1,711 ( 92.9%)
  Problèmes résolus  :   267 ( 98.9%)


In [13]:
# Afficher le tableau complet
print("\n" + "="*80)
print("📊 TABLEAU RÉCAPITULATIF")
print("="*80)
display(df_results.style.format({
    'Seuil_h': '{:.1f}h',
    'Locations_bloquees': '{:,.0f}',
    'Pct_bloquees': '{:.1f}%',
    'Problemes_resolus': '{:,.0f}',
    'Pct_resolus': '{:.1f}%'
}))


📊 TABLEAU RÉCAPITULATIF


,Seuil_min,Seuil_h,Locations_bloquees,Pct_bloquees,Problemes_resolus,Pct_resolus
0,0,0.0h,0,0.0%,0,0.0%
1,30,0.5h,279,15.2%,136,50.4%
2,60,1.0h,401,21.8%,176,65.2%
3,120,2.0h,666,36.2%,224,83.0%
4,180,3.0h,870,47.3%,245,90.7%
5,240,4.0h,"1,001",54.4%,251,93.0%
6,360,6.0h,"1,170",63.6%,255,94.4%
7,480,8.0h,"1,290",70.1%,259,95.9%
8,720,12.0h,"1,711",92.9%,267,98.9%


### 5.2 Graphique Trade-off

In [14]:
# Graphique interactif du trade-off
fig = go.Figure()

# Courbe 1 : Locations bloquées (impact négatif)
fig.add_trace(go.Scatter(
    x=df_results['Seuil_h'],
    y=df_results['Pct_bloquees'],
    mode='lines+markers',
    name='Locations bloquées (%)',
    line=dict(color='red', width=3),
    marker=dict(size=10),
    hovertemplate='<b>Seuil</b>: %{x:.1f}h<br><b>Bloquées</b>: %{y:.1f}%<extra></extra>'
))

# Courbe 2 : Problèmes résolus (impact positif)
fig.add_trace(go.Scatter(
    x=df_results['Seuil_h'],
    y=df_results['Pct_resolus'],
    mode='lines+markers',
    name='Problèmes résolus (%)',
    line=dict(color='green', width=3),
    marker=dict(size=10),
    hovertemplate='<b>Seuil</b>: %{x:.1f}h<br><b>Résolus</b>: %{y:.1f}%<extra></extra>'
))

fig.update_layout(
    title='🎯 Trade-off : Locations bloquées vs Problèmes résolus',
    xaxis_title='Seuil minimum (heures)',
    yaxis_title='Pourcentage (%)',
    hovermode='x unified',
    height=500,
    template='plotly_white',
    legend=dict(x=0.7, y=0.5)
)

fig.show()

## 6. Recommandations

In [15]:
# Identifier le seuil optimal (exemple : meilleur ratio problèmes résolus / locations bloquées)
df_results['ratio'] = df_results['Pct_resolus'] / (df_results['Pct_bloquees'] + 1)  # +1 pour éviter division par 0
optimal_idx = df_results['ratio'].idxmax()
optimal_threshold = df_results.loc[optimal_idx]

print("="*80)
print("💡 RECOMMANDATIONS")
print("="*80)

print(f"\n🎯 Seuil optimal suggéré : {optimal_threshold['Seuil_min']:.0f} minutes ({optimal_threshold['Seuil_h']:.1f} heures)")
print(f"\n📊 Impact attendu avec ce seuil :")
print(f"   ✅ Problèmes résolus : {optimal_threshold['Problemes_resolus']:.0f} ({optimal_threshold['Pct_resolus']:.1f}%)")
print(f"   ⚠️  Locations bloquées : {optimal_threshold['Locations_bloquees']:.0f} ({optimal_threshold['Pct_bloquees']:.1f}%)")

print(f"\n🔍 Insights clés :")
print(f"   - {late_pct:.1f}% des locations sont en retard")
print(f"   - {problem_pct:.2f}% des locations consécutives ont des problèmes")
print(f"   - Le type '{df_checkin.loc[df_checkin['Pct_retards'].idxmax(), 'Type']}' a le plus de retards")

print(f"\n💼 Recommandation de périmètre :")
print(f"   - Commencer avec les voitures 'Connect' uniquement")
print(f"   - Étendre progressivement selon les résultats")

💡 RECOMMANDATIONS

🎯 Seuil optimal suggéré : 30 minutes (0.5 heures)

📊 Impact attendu avec ce seuil :
   ✅ Problèmes résolus : 136 (50.4%)
   ⚠️  Locations bloquées : 279 (15.2%)

🔍 Insights clés :
   - 33.9% des locations sont en retard
   - 14.67% des locations consécutives ont des problèmes
   - Le type 'mobile' a le plus de retards

💼 Recommandation de périmètre :
   - Commencer avec les voitures 'Connect' uniquement
   - Étendre progressivement selon les résultats


## 7. Sauvegarde des insights

In [16]:
# Sauvegarder les résultats de simulation pour le dashboard
df_results.to_csv('../data/threshold_simulation_results.csv', index=False)
print("✅ Résultats sauvegardés dans 'data/threshold_simulation_results.csv'")

✅ Résultats sauvegardés dans 'data/threshold_simulation_results.csv'


## 📝 Conclusions

### Points clés à retenir :

#### 1. **Fréquence des retards**
- **44.1%** des locations (9,404 sur 21,310) sont en retard au checkout
- Le retard moyen est de **201.8 minutes** (3.4 heures)
- Le retard médian est de **53.0 minutes** (0.9 heure)
- 🔴 Le type **Mobile** a le plus de retards : **46.7%** (retard moyen : 224.1 min)
- 🟢 Le type **Connect** est plus performant : **33.9%** de retards (retard moyen : 80.1 min)

#### 2. **Impact sur les clients suivants**
- Seulement **8.6%** des locations (1,841) ont une location suivante immédiate
- Parmi ces locations consécutives, **14.67%** (270 cas) sont **problématiques**
- Ces 270 cas représentent des situations où le client suivant a été directement impacté par un retard

#### 3. **Seuil optimal recommandé**
- **Seuil suggéré : 30 minutes (0.5h)**
- Ce seuil offre le meilleur ratio bénéfice/coût :
  - ✅ Résout **50.4%** des problèmes (136 cas sur 270)
  - ⚠️ Bloque **15.2%** des locations consécutives (279 locations)
  
**Alternative : Seuil de 120 minutes (2h)**
- Plus conservateur mais très efficace :
  - ✅ Résout **83.0%** des problèmes (224 cas)
  - ⚠️ Bloque **36.2%** des locations (666 locations)

#### 4. **Trade-off**
Le graphique montre clairement que :
- Entre **30 min et 120 min** : zone optimale avec le meilleur rapport efficacité/impact
- Au-delà de **180 min** : gains marginaux décroissants (on résout peu de problèmes supplémentaires mais on bloque beaucoup de locations)
- En dessous de **30 min** : protection insuffisante

### 💡 Recommandations finales

#### Périmètre d'application :
1. **Phase 1** : Commencer avec les voitures **Connect uniquement**
   - Elles ont déjà moins de retards (33.9% vs 46.7%)
   - Impact moindre sur le revenu
   - Test terrain plus contrôlé

2. **Phase 2** : Si résultats positifs, étendre aux voitures **Mobile**

#### Seuil recommandé selon le profil de risque :
- **Conservateur** : 30 minutes - Bloque peu, résout la moitié des problèmes
- **Équilibré** : 60-120 minutes - Bon compromis (65-83% de résolution)
- **Agressif** : 180 minutes - Très protecteur mais impact significatif sur revenus

### 📊 Métriques de succès à suivre :
1. Réduction des plaintes clients pour "voiture non disponible"
2. Taux d'annulation des locations suivantes
3. Impact réel sur le revenu des propriétaires
4. Satisfaction client (NPS)
5. Taux de retard après mise en place